# 机器实战 ----SVM 手写字体识别

In [1]:
from numpy import *
from time import sleep
def loadDataSet(fileName):
    dataMat = []; labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t') # 去掉空格符
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2])) # 标签
    return dataMat,labelMat
def selectJrand(i,m):
    j=i #we want to select any J not equal to i
    while (j==i):
        j = int(random.uniform(0,m))
    return j

def clipAlpha(aj,H,L):
    if aj > H: 
        aj = H
    if L > aj:
        aj = L
    return aj

In [2]:
dataArr,labelArr=loadDataSet('testSet.txt')
dataArr

[[3.542485, 1.977398],
 [3.018896, 2.556416],
 [7.55151, -1.58003],
 [2.114999, -0.004466],
 [8.127113, 1.274372],
 [7.108772, -0.986906],
 [8.610639, 2.046708],
 [2.326297, 0.265213],
 [3.634009, 1.730537],
 [0.341367, -0.894998],
 [3.125951, 0.293251],
 [2.123252, -0.783563],
 [0.887835, -2.797792],
 [7.139979, -2.329896],
 [1.696414, -1.212496],
 [8.117032, 0.623493],
 [8.497162, -0.266649],
 [4.658191, 3.507396],
 [8.197181, 1.545132],
 [1.208047, 0.2131],
 [1.928486, -0.32187],
 [2.175808, -0.014527],
 [7.886608, 0.461755],
 [3.223038, -0.552392],
 [3.628502, 2.190585],
 [7.40786, -0.121961],
 [7.286357, 0.251077],
 [2.301095, -0.533988],
 [-0.232542, -0.54769],
 [3.457096, -0.082216],
 [3.023938, -0.057392],
 [8.015003, 0.885325],
 [8.991748, 0.923154],
 [7.916831, -1.781735],
 [7.616862, -0.217958],
 [2.450939, 0.744967],
 [7.270337, -2.507834],
 [1.749721, -0.961902],
 [1.803111, -0.176349],
 [8.804461, 3.044301],
 [1.231257, -0.568573],
 [2.074915, 1.41055],
 [-0.743036, -1.73

In [6]:
#  完整版的SMO 算法 外循环代码
def smoP(dataMatIn, classLabels, C, toler, maxIter,kTup=('lin', 0)):    #full Platt SMO
    oS = optStruct(mat(dataMatIn),mat(classLabels).transpose(),C,toler, kTup)
    iter = 0
    entireSet = True; alphaPairsChanged = 0
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)): # 判断循环停止条件
        alphaPairsChanged = 0
        if entireSet:   #go over all
            for i in range(oS.m):        
                alphaPairsChanged += innerL(i,oS)
                print("fullSet, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
            iter += 1
        else:#go over non-bound (railed) alphas
            nonBoundIs = nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i,oS)
                print("non-bound, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
            iter += 1
        if entireSet: entireSet = False #toggle entire set loop
        elif (alphaPairsChanged == 0): entireSet = True  
        print("iteration number: %d" % iter)
    return oS.b,oS.alphas
class optStruct: # 采用数据结构来存储数据对象 ，kTup 包含核函数信息元组
    def __init__(self,dataMatIn, classLabels, C, toler, kTup):  # Initialize the structure with the parameters 
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m,1)))
        self.b = 0
        self.eCache = mat(zeros((self.m,2))) #first column is valid flag
        self.K = mat(zeros((self.m,self.m)))
        for i in range(self.m):
            self.K[:,i] = kernelTrans(self.X, self.X[i,:], kTup) # 核函数的转换

def calcEk(oS, k): # 计算E值并返回
    fXk = float(multiply(oS.alphas,oS.labelMat).T*oS.K[:,k] + oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek
        
def selectJ(i, oS, Ei):         #this is the second choice -heurstic, and calcs Ej
    maxK = -1; maxDeltaE = 0; Ej = 0
    oS.eCache[i] = [1,Ei]  #set valid #choose the alpha that gives the maximum delta E
    validEcacheList = nonzero(oS.eCache[:,0].A)[0]
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:   #loop through valid Ecache values and find the one that maximizes delta E
            if k == i: continue #don't calc for i, waste of time
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):
                maxK = k; maxDeltaE = deltaE; Ej = Ek
        return maxK, Ej
    else:   #in this case (first time around) we don't have any valid eCache values
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej

def updateEk(oS, k):#after any alpha has changed update the new value in the cache
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1,Ek]
# 核函数转换 转换到高维空间中
def kernelTrans(X, A, kTup): #calc the kernel or transform data to a higher dimensional space
    m,n = shape(X)
    K = mat(zeros((m,1)))
    if kTup[0]=='lin': K = X * A.T   #线性核
    elif kTup[0]=='rbf':
        for j in range(m):
            deltaRow = X[j,:] - A
            K[j] = deltaRow*deltaRow.T
        K = exp(K/(-1*kTup[1]**2)) #divide in NumPy is element-wise not matrix like Matlab
    else: raise NameError('Houston We Have a Problem -- \
    That Kernel is not recognized')
    return K

In [7]:
#完整版的Platt SMO 算法优化例程
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i] > 0)):
        j,Ej = selectJ(i, oS, Ei) #this has been changed from selectJrand
        alphaIold = oS.alphas[i].copy(); alphaJold = oS.alphas[j].copy();
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H: print("L==H"); return 0
        eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,j] #changed for kernel
        if eta >= 0: print("eta>=0"); return 0
        oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS, j) #added this for the Ecache
        if (abs(oS.alphas[j] - alphaJold) < 0.00001): print("j not moving enough"); return 0
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold - oS.alphas[j])#update i by the same amount as j
        updateEk(oS, i) #added this for the Ecache                    #the update is in the oppostie direction
        b1 = oS.b - Ei- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i] - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j]
        b2 = oS.b - Ej- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,j]- oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
        else: oS.b = (b1 + b2)/2.0
        return 1
    else: return 0

In [8]:
b,aplhas=smoP(dataArr,labelArr,0.6,0.001,40)

L==H
fullSet, iter: 0 i:0, pairs changed 0
L==H
fullSet, iter: 0 i:1, pairs changed 0
fullSet, iter: 0 i:2, pairs changed 1
L==H
fullSet, iter: 0 i:3, pairs changed 1
fullSet, iter: 0 i:4, pairs changed 2
fullSet, iter: 0 i:5, pairs changed 2
fullSet, iter: 0 i:6, pairs changed 2
j not moving enough
fullSet, iter: 0 i:7, pairs changed 2
L==H
fullSet, iter: 0 i:8, pairs changed 2
fullSet, iter: 0 i:9, pairs changed 2
L==H
fullSet, iter: 0 i:10, pairs changed 2
L==H
fullSet, iter: 0 i:11, pairs changed 2
L==H
fullSet, iter: 0 i:12, pairs changed 2
fullSet, iter: 0 i:13, pairs changed 2
L==H
fullSet, iter: 0 i:14, pairs changed 2
fullSet, iter: 0 i:15, pairs changed 2
fullSet, iter: 0 i:16, pairs changed 2
L==H
fullSet, iter: 0 i:17, pairs changed 2
fullSet, iter: 0 i:18, pairs changed 3
fullSet, iter: 0 i:19, pairs changed 3
fullSet, iter: 0 i:20, pairs changed 3
fullSet, iter: 0 i:21, pairs changed 3
j not moving enough
fullSet, iter: 0 i:22, pairs changed 3
L==H
fullSet, iter: 0 i:23, 

In [10]:
#进行分类
def  calcWs(alphas,dataArr,labelArr):
    X=mat(dataArr)
    labelMat=mat(labelArr).transpose()
    m,n=shape(X)
    w=zeros((n,1))
    for i in range(m):
        w+=multiply(alphas[i]*labelMat[i],X[i,:].T)
    return w
ws=calcWs(aplhas,dataArr,labelArr)
ws

array([[ 0.65307162],
       [-0.17196128]])

In [12]:
# 对数据分类，第一个数据点
dataMat=mat(dataArr)
label=dataMat[0]*mat(ws)+b
if label>0:
    label=1
else :
    label=-1
if label==labelArr[0]:
    print("分类结果正确")

分类结果正确


In [13]:
# 测试核函数
def testRbf(k1=1.3):
    dataArr,labelArr = loadDataSet('testSetRBF.txt')
    b,alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, ('rbf', k1)) #C=200 important
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    svInd=nonzero(alphas.A>0)[0] # alphas.A值得是什么
    sVs=datMat[svInd] #get matrix of only support vectors
    labelSV = labelMat[svInd];
    print("there are %d Support Vectors" % shape(sVs)[0])
    m,n = shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],('rbf', k1))
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1
    print("the training error rate is: %f" % (float(errorCount)/m))
    dataArr,labelArr = loadDataSet('testSetRBF2.txt')
    errorCount = 0
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    m,n = shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],('rbf', k1))
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1    
    print("the test error rate is: %f" % (float(errorCount)/m))

In [14]:
testRbf(k1=1.3)

fullSet, iter: 0 i:0, pairs changed 1
fullSet, iter: 0 i:1, pairs changed 2
fullSet, iter: 0 i:2, pairs changed 3
fullSet, iter: 0 i:3, pairs changed 4
fullSet, iter: 0 i:4, pairs changed 5
fullSet, iter: 0 i:5, pairs changed 6
fullSet, iter: 0 i:6, pairs changed 6
fullSet, iter: 0 i:7, pairs changed 6
fullSet, iter: 0 i:8, pairs changed 7
fullSet, iter: 0 i:9, pairs changed 7
L==H
fullSet, iter: 0 i:10, pairs changed 7
fullSet, iter: 0 i:11, pairs changed 8
L==H
fullSet, iter: 0 i:12, pairs changed 8
L==H
fullSet, iter: 0 i:13, pairs changed 8
L==H
fullSet, iter: 0 i:14, pairs changed 8
fullSet, iter: 0 i:15, pairs changed 9
fullSet, iter: 0 i:16, pairs changed 10
L==H
fullSet, iter: 0 i:17, pairs changed 10
fullSet, iter: 0 i:18, pairs changed 11
fullSet, iter: 0 i:19, pairs changed 12
fullSet, iter: 0 i:20, pairs changed 12
fullSet, iter: 0 i:21, pairs changed 13
fullSet, iter: 0 i:22, pairs changed 13
fullSet, iter: 0 i:23, pairs changed 14
fullSet, iter: 0 i:24, pairs changed 15
f

L==H
fullSet, iter: 5 i:58, pairs changed 0
fullSet, iter: 5 i:59, pairs changed 0
fullSet, iter: 5 i:60, pairs changed 0
fullSet, iter: 5 i:61, pairs changed 0
j not moving enough
fullSet, iter: 5 i:62, pairs changed 0
fullSet, iter: 5 i:63, pairs changed 0
fullSet, iter: 5 i:64, pairs changed 0
L==H
fullSet, iter: 5 i:65, pairs changed 0
fullSet, iter: 5 i:66, pairs changed 0
fullSet, iter: 5 i:67, pairs changed 0
fullSet, iter: 5 i:68, pairs changed 0
fullSet, iter: 5 i:69, pairs changed 0
fullSet, iter: 5 i:70, pairs changed 0
fullSet, iter: 5 i:71, pairs changed 0
fullSet, iter: 5 i:72, pairs changed 0
fullSet, iter: 5 i:73, pairs changed 0
j not moving enough
fullSet, iter: 5 i:74, pairs changed 0
fullSet, iter: 5 i:75, pairs changed 0
L==H
fullSet, iter: 5 i:76, pairs changed 0
fullSet, iter: 5 i:77, pairs changed 0
L==H
fullSet, iter: 5 i:78, pairs changed 0
fullSet, iter: 5 i:79, pairs changed 0
fullSet, iter: 5 i:80, pairs changed 0
fullSet, iter: 5 i:81, pairs changed 0
L==H

# 手写识别数字问题

In [19]:
# 基于SVM 的手写数字识别
def img2vector(filename):
    returnVect = zeros((1,1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0,32*i+j] = int(lineStr[j])
    return returnVect

def loadImages(dirName):
    from os import listdir
    hwLabels = []
    trainingFileList = listdir(dirName)           #load the training set
    m = len(trainingFileList)
    trainingMat = zeros((m,1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]     #take off .txt
        classNumStr = int(fileStr.split('_')[0])
        if classNumStr == 9: hwLabels.append(-1)
        else: hwLabels.append(1)
        trainingMat[i,:] = img2vector('%s/%s' % (dirName, fileNameStr))
    return trainingMat, hwLabels    

def testDigits(kTup=('rbf', 10)):
    dataArr,labelArr = loadImages('./digits/trainingDigits')
    b,alphas = smoP(dataArr, labelArr, 200, 0.0005, 10000, kTup)
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    svInd=nonzero(alphas.A>0)[0]
    sVs=datMat[svInd] 
    labelSV = labelMat[svInd];
    print("there are %d Support Vectors" % shape(sVs)[0])
    m,n = shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1
    print("the training error rate is: %f" % (float(errorCount)/m))
    dataArr,labelArr = loadImages('./digits/testDigits')
    errorCount = 0
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    m,n = shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1    
    print("the test error rate is: %f" % (float(errorCount)/m))

In [22]:

testDigits(('rbf', 25))

fullSet, iter: 0 i:0, pairs changed 1
fullSet, iter: 0 i:1, pairs changed 2
fullSet, iter: 0 i:2, pairs changed 3
fullSet, iter: 0 i:3, pairs changed 4
fullSet, iter: 0 i:4, pairs changed 5
fullSet, iter: 0 i:5, pairs changed 6
fullSet, iter: 0 i:6, pairs changed 6
fullSet, iter: 0 i:7, pairs changed 7
fullSet, iter: 0 i:8, pairs changed 8
fullSet, iter: 0 i:9, pairs changed 8
fullSet, iter: 0 i:10, pairs changed 9
fullSet, iter: 0 i:11, pairs changed 9
fullSet, iter: 0 i:12, pairs changed 10
fullSet, iter: 0 i:13, pairs changed 11
L==H
fullSet, iter: 0 i:14, pairs changed 11
L==H
fullSet, iter: 0 i:15, pairs changed 11
fullSet, iter: 0 i:16, pairs changed 11
fullSet, iter: 0 i:17, pairs changed 11
fullSet, iter: 0 i:18, pairs changed 11
fullSet, iter: 0 i:19, pairs changed 11
fullSet, iter: 0 i:20, pairs changed 11
L==H
fullSet, iter: 0 i:21, pairs changed 11
fullSet, iter: 0 i:22, pairs changed 11
fullSet, iter: 0 i:23, pairs changed 11
fullSet, iter: 0 i:24, pairs changed 11
fullSet

L==H
fullSet, iter: 0 i:279, pairs changed 22
fullSet, iter: 0 i:280, pairs changed 22
L==H
fullSet, iter: 0 i:281, pairs changed 22
L==H
fullSet, iter: 0 i:282, pairs changed 22
fullSet, iter: 0 i:283, pairs changed 22
L==H
fullSet, iter: 0 i:284, pairs changed 22
fullSet, iter: 0 i:285, pairs changed 22
fullSet, iter: 0 i:286, pairs changed 22
L==H
fullSet, iter: 0 i:287, pairs changed 22
L==H
fullSet, iter: 0 i:288, pairs changed 22
L==H
fullSet, iter: 0 i:289, pairs changed 22
L==H
fullSet, iter: 0 i:290, pairs changed 22
L==H
fullSet, iter: 0 i:291, pairs changed 22
L==H
fullSet, iter: 0 i:292, pairs changed 22
L==H
fullSet, iter: 0 i:293, pairs changed 22
fullSet, iter: 0 i:294, pairs changed 22
fullSet, iter: 0 i:295, pairs changed 22
L==H
fullSet, iter: 0 i:296, pairs changed 22
L==H
fullSet, iter: 0 i:297, pairs changed 22
L==H
fullSet, iter: 0 i:298, pairs changed 22
fullSet, iter: 0 i:299, pairs changed 22
fullSet, iter: 0 i:300, pairs changed 22
L==H
fullSet, iter: 0 i:301,

j not moving enough
non-bound, iter: 3 i:287, pairs changed 1
j not moving enough
non-bound, iter: 3 i:297, pairs changed 1
j not moving enough
non-bound, iter: 3 i:311, pairs changed 1
j not moving enough
non-bound, iter: 3 i:317, pairs changed 1
j not moving enough
non-bound, iter: 3 i:318, pairs changed 1
j not moving enough
non-bound, iter: 3 i:322, pairs changed 1
j not moving enough
non-bound, iter: 3 i:323, pairs changed 1
j not moving enough
non-bound, iter: 3 i:325, pairs changed 1
j not moving enough
non-bound, iter: 3 i:330, pairs changed 1
j not moving enough
non-bound, iter: 3 i:332, pairs changed 1
j not moving enough
non-bound, iter: 3 i:334, pairs changed 1
j not moving enough
non-bound, iter: 3 i:337, pairs changed 1
j not moving enough
non-bound, iter: 3 i:343, pairs changed 1
j not moving enough
non-bound, iter: 3 i:349, pairs changed 1
j not moving enough
non-bound, iter: 3 i:362, pairs changed 1
j not moving enough
non-bound, iter: 3 i:370, pairs changed 1
j not mo

j not moving enough
fullSet, iter: 5 i:113, pairs changed 1
fullSet, iter: 5 i:114, pairs changed 1
L==H
fullSet, iter: 5 i:115, pairs changed 1
j not moving enough
fullSet, iter: 5 i:116, pairs changed 1
L==H
fullSet, iter: 5 i:117, pairs changed 1
fullSet, iter: 5 i:118, pairs changed 1
L==H
fullSet, iter: 5 i:119, pairs changed 1
L==H
fullSet, iter: 5 i:120, pairs changed 1
fullSet, iter: 5 i:121, pairs changed 1
fullSet, iter: 5 i:122, pairs changed 1
j not moving enough
fullSet, iter: 5 i:123, pairs changed 1
L==H
fullSet, iter: 5 i:124, pairs changed 1
L==H
fullSet, iter: 5 i:125, pairs changed 1
fullSet, iter: 5 i:126, pairs changed 1
j not moving enough
fullSet, iter: 5 i:127, pairs changed 1
fullSet, iter: 5 i:128, pairs changed 1
L==H
fullSet, iter: 5 i:129, pairs changed 1
j not moving enough
fullSet, iter: 5 i:130, pairs changed 1
j not moving enough
fullSet, iter: 5 i:131, pairs changed 1
fullSet, iter: 5 i:132, pairs changed 1
fullSet, iter: 5 i:133, pairs changed 1
fullS

j not moving enough
fullSet, iter: 5 i:362, pairs changed 1
fullSet, iter: 5 i:363, pairs changed 1
fullSet, iter: 5 i:364, pairs changed 1
fullSet, iter: 5 i:365, pairs changed 1
fullSet, iter: 5 i:366, pairs changed 1
fullSet, iter: 5 i:367, pairs changed 1
fullSet, iter: 5 i:368, pairs changed 1
fullSet, iter: 5 i:369, pairs changed 1
j not moving enough
fullSet, iter: 5 i:370, pairs changed 1
fullSet, iter: 5 i:371, pairs changed 1
fullSet, iter: 5 i:372, pairs changed 1
fullSet, iter: 5 i:373, pairs changed 1
fullSet, iter: 5 i:374, pairs changed 1
fullSet, iter: 5 i:375, pairs changed 1
fullSet, iter: 5 i:376, pairs changed 1
fullSet, iter: 5 i:377, pairs changed 1
fullSet, iter: 5 i:378, pairs changed 1
fullSet, iter: 5 i:379, pairs changed 1
fullSet, iter: 5 i:380, pairs changed 1
fullSet, iter: 5 i:381, pairs changed 1
fullSet, iter: 5 i:382, pairs changed 1
fullSet, iter: 5 i:383, pairs changed 1
fullSet, iter: 5 i:384, pairs changed 1
fullSet, iter: 5 i:385, pairs changed 1


j not moving enough
fullSet, iter: 7 i:173, pairs changed 0
fullSet, iter: 7 i:174, pairs changed 0
fullSet, iter: 7 i:175, pairs changed 0
fullSet, iter: 7 i:176, pairs changed 0
L==H
fullSet, iter: 7 i:177, pairs changed 0
fullSet, iter: 7 i:178, pairs changed 0
L==H
fullSet, iter: 7 i:179, pairs changed 0
fullSet, iter: 7 i:180, pairs changed 0
fullSet, iter: 7 i:181, pairs changed 0
j not moving enough
fullSet, iter: 7 i:182, pairs changed 0
j not moving enough
fullSet, iter: 7 i:183, pairs changed 0
fullSet, iter: 7 i:184, pairs changed 0
j not moving enough
fullSet, iter: 7 i:185, pairs changed 0
j not moving enough
fullSet, iter: 7 i:186, pairs changed 0
fullSet, iter: 7 i:187, pairs changed 0
j not moving enough
fullSet, iter: 7 i:188, pairs changed 0
j not moving enough
fullSet, iter: 7 i:189, pairs changed 0
fullSet, iter: 7 i:190, pairs changed 0
fullSet, iter: 7 i:191, pairs changed 0
fullSet, iter: 7 i:192, pairs changed 0
fullSet, iter: 7 i:193, pairs changed 0
fullSet, i

the training error rate is: 0.000000
the test error rate is: 0.010753
